<a href="https://colab.research.google.com/github/siddsh/Information_Retieval/blob/master/IR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

     |████████████████████████████████| 1.8MB 4.5MB/s 
     |████████████████████████████████| 17.3MB 174kB/s 
     |████████████████████████████████| 245kB 48.3MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [3]:
#CHECK IF THE Cuda version is the same as 10.1 if not change the mxnet-cuXXXmkl
import torch
torch.version.cuda

'10.1'

In [2]:
pip install mxnet-cu101mkl

     |████████████████████████████████| 712.3MB 26kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [11]:
!pip install bert_embedding

     |████████████████████████████████| 29.6MB 103kB/s 
     |████████████████████████████████| 215kB 51.4MB/s 
     |████████████████████████████████| 13.8MB 245kB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.6.0-cp36-none-any.whl size=259917 sha256=c67916bad291ff2690135f5b76a98c4c9a28b0d13f17fd715edc35d498a130a0
  Stored in directory: /root/.cache/pip/wheels/ff/48/ac/a77c79aa416ba6dd7bf487f2280b0471034f66141617965914
Successfully built gluonnlp
ERROR: xarray 0.15.1 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have numpy 1.14.6 which is incompatible.
ERROR: tifffile 2020.9.3 has requirement numpy>=1.15.1, but you'll have numpy 1.14.6 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: spacy 2.2.4 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: pl

In [5]:
import mxnet as mx
ctx = mx.gpu(0)

In [6]:
from datasets import load_dataset

dataset = load_dataset("aeslc")

Using custom data configuration default


Dataset aeslc downloaded and prepared to /root/.cache/huggingface/datasets/aeslc/default/1.0.0/0f335d7ea7f9ab9e325673a86c6aa0e2cd503c5dbf17d48fddcb02840a92fc96. Subsequent calls will reuse this data.


In [7]:
#Emails as string
s = dataset['train'][:]['email_body']
len(s)

14436

In [76]:
#Emails as 
arr = []
for i in range(len(s)):
    arr.append([i for i in s[i].split('\n') if len(i)!=0 ])
print(len(arr))

14436


In [12]:
#RUN ON GPU
from bert_embedding import BertEmbedding
be=BertEmbedding(ctx=ctx)
#If no GPU Then run the next line
#be=BertEmbedding()

Vocab file is not found. Downloading.


In [ ]:
emb = None
for email in range(len(arr)):
    emb = be(arr[email])
    emb = emb
    print(emb)
    break

In [15]:
ad = []
for email in range(len(arr)):
    printc=True
    for i in be(arr[email]):
        if email%500==0 and printc:
            print("Iteration: ", email)
            printc=False
        try:
            ad.append(sum(i[1])/len(i[1]))
        except:
            pass
print(len(ad), len(arr[0])) 

Iteration:  0
Iteration:  50
Iteration:  100
Iteration:  150
Iteration:  200
Iteration:  250
Iteration:  300
Iteration:  350
Iteration:  400
Iteration:  450
Iteration:  500
Iteration:  550
Iteration:  600
Iteration:  650
Iteration:  700
Iteration:  750
Iteration:  800
Iteration:  850
Iteration:  900
Iteration:  950
Iteration:  1000
Iteration:  1050
Iteration:  1100
Iteration:  1150
Iteration:  1200
Iteration:  1250
Iteration:  1300
Iteration:  1350
Iteration:  1400
Iteration:  1450
Iteration:  1500
Iteration:  1550
Iteration:  1600
Iteration:  1650
Iteration:  1700
Iteration:  1750
Iteration:  1800
Iteration:  1850
Iteration:  1900
Iteration:  1950
Iteration:  2000
Iteration:  2050
Iteration:  2100
Iteration:  2150
Iteration:  2200
Iteration:  2250
Iteration:  2300
Iteration:  2350
Iteration:  2400
Iteration:  2450
Iteration:  2500
Iteration:  2550
Iteration:  2600
Iteration:  2650
Iteration:  2700
Iteration:  2750
Iteration:  2800
Iteration:  2850
Iteration:  2900
Iteration:  2950
Ite

In [16]:
#COUNT OF SEntences and Email
print(len(ad), len(arr)) 

106257 14436


In [17]:
#Count sentences in email corpus
l=0
for i in arr:
    l+=len(i)
print(l)

106257


In [22]:
#Save Data to PKL file
import pickle

with open('Index.pkl', 'wb') as fp:
    pickle.dump(ad, fp)    

In [21]:
#Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [39]:
#Copy file to Drive
!sudo cp "/content/Index.pkl" '/content/drive/My Drive/Index.pkl' -f

In [91]:
with open ('/content/drive/My Drive/Index.pkl', 'rb') as fp:
    itemlist = pickle.load(fp)
len(itemlist)

106257

In [222]:
#Check if Index.pkl is correctly loaded
assert len([False for i in ad[0] == itemlist[0] if i is False]) ==0

In [224]:
l=0 #INDEX THROUGH EVERY SENTENCE
email_emb=[] #VEC OF EMAIL
for email in arr:
    sarr = [] #SENTENCE VECS
    for _ in email:
        sarr.append(itemlist[l])
        l+=1
    email_emb.append(sum(sarr)/len(sarr))
len(sarr), l

(4, 106257)

In [225]:
len(email_emb), len(arr)

(14436, 14436)

In [103]:
#Save Email Embeddings
with open('Email_Embeddings.pkl', 'wb') as fp:
    pickle.dump(ad, fp) 
!sudo cp "/content/Email_Embeddings.pkl" '/content/drive/My Drive/Email_Embeddings.pkl' -f

In [226]:
import textwrap
wrapper = textwrap.TextWrapper(width=180) 
def wrapp(string):
    '''
    Print the String by wrapping
    '''
    string=str(string)
    word_list = wrapper.wrap(text=string) 
  
    # Print each line. 
    for element in word_list: 
        print(element)

def search(query):
    '''
    Take query and print it
    '''
    embs = be(query)
    q_emb = sum(embs[0][1])/len(embs[0][1])
    import numpy as np
    res = []
    for i in range(len(email_emb)):
        res.append(
            np.dot(q_emb, email_emb[i])/
            (np.linalg.norm(q_emb) * np.linalg.norm(email_emb[i])
        )
    )
    indx = np.argsort(res)[-1:-6:-1]
    print(query, "\n\n")
    for i in indx:
        wrapp(res[i])
        wrapp(dataset['train']['subject_line'][i])
        wrapp(s[i])
        print()

In [227]:
query = ["""Apply for this job and i have sent you my resume. Please consider me for this post"""]
search(query)

['Apply for this job and i have sent you my resume. Please consider me for this post'] 


0.82568026
Penn Career Network
Mr. Shankman:  My name is Robert Brown and I am a fellow alumnus of the Wharton School. I noticed that you provided your name on the Penn Career Network as a mentor for other
alumni. I have just returned from a sabbatical and am currently seeking to enter the energy industry in Houston now that I have returned stateside. As a person who has achieved a
prominent position in the industry in which I am interested, I thought you might be able to help me in my endeavors with some advice. Would it be possible for me to contact you at
a convenient time, so that I might be able to ask you a few questions? Please feel free to call me on my cell phone at 832.541.5805 or you can also reach me through e-mail by
responding to this e-mail. Thank you for making yourself available to other alumni like myself. I look forward to speaking to you. Regards,

0.82326734
December Graduates